In [60]:
from collections import defaultdict
import copy

In [1]:
train = [
    (1, 1, 1, 1, 1, 1, 1, 1, 1, 'a'),
    (1, 1, 2, 2, 1, 1, 2, 1, 1, 'a'),
    (2, 1, 2, 2, 1, 1, 3, 2, 4, 'a'),
    (2, 1, 2, 4, 1, 1, 3, 2, 4, 'a'),
    (1, 2, 1, 1, 2, 1, 1, 2, 1, 'b'),
    (1, 2, 3, 3, 2, 1, 3, 1, 1, 'b'),
    (2, 2, 1, 2, 1, 2, 4, 2, 2, 'b'),
    (2, 3, 4, 4, 1, 1, 3, 2, 2, 'c'),
    (2, 1, 4, 4, 1, 1, 4, 2, 3, 'c'),
    (3, 4, 5, 3, 3, 3, 1, 3, 4, 'd'),
    (4, 5, 5, 5, 2, 2, 4, 2, 1, 'd'),
    (3, 5, 5, 1, 2, 1, 5, 2, 5, 'd'),
    (3, 4, 3, 5, 4, 4, 2, 4, 1, 'e'),
    (4, 2, 3, 3, 2, 4, 2, 2, 1, 'e')
]

In [58]:
train_dict = defaultdict(list)
for x in train:
    train_dict[x[-1]].append(x[:-1])

In [59]:
train_dict

defaultdict(list,
            {'a': [(1, 1, 1, 1, 1, 1, 1, 1, 1),
              (1, 1, 2, 2, 1, 1, 2, 1, 1),
              (2, 1, 2, 2, 1, 1, 3, 2, 4),
              (2, 1, 2, 4, 1, 1, 3, 2, 4)],
             'b': [(1, 2, 1, 1, 2, 1, 1, 2, 1),
              (1, 2, 3, 3, 2, 1, 3, 1, 1),
              (2, 2, 1, 2, 1, 2, 4, 2, 2)],
             'c': [(2, 3, 4, 4, 1, 1, 3, 2, 2), (2, 1, 4, 4, 1, 1, 4, 2, 3)],
             'd': [(3, 4, 5, 3, 3, 3, 1, 3, 4),
              (4, 5, 5, 5, 2, 2, 4, 2, 1),
              (3, 5, 5, 1, 2, 1, 5, 2, 5)],
             'e': [(3, 4, 3, 5, 4, 4, 2, 4, 1), (4, 2, 3, 3, 2, 4, 2, 2, 1)]})

In [57]:
def kw(x, y):
    assert len(x) == len(y)
    count = 0
    for i in range(len(x)):
        if x[i] != y[i]:
            count += 1
    return count

def mw(attr, d_plus, Q_j):
    count = 0
    for d_minus in Q_j:
        if d_plus[attr] != d_minus[attr]:
            count += 1
    return count

In [23]:
def distinguishing(x, y):
    assert len(x) == len(y)
    for i in range(len(x)):
        if x[i] == y[i]:
            continue
        yield i

In [168]:
def poisk(d_plus, Q_j_orig):
    t = set()
    Q_j = copy.deepcopy(Q_j_orig)
    if d_plus in Q_j:
        Q_j.remove(d_plus)
    _kw = {d_minus: kw(d_plus, d_minus) for d_minus in Q_j}
    _mw = {i: mw(i, d_plus, Q_j) for i in range(len(d_plus))}
    while Q_j:
        d_minus_selected = min(_kw, key=_kw.get)
        attr_selected = max(_mw, key=_mw.get)
        t.add(attr_selected)
        for attr in distinguishing(d_plus, d_minus_selected):
            _mw[attr] -= 1
        Q_j.remove(d_minus_selected)
        _kw.pop(d_minus_selected)
    return t

In [169]:
def my_poisk(d_plus, Q_j_orig):
    t = set()
    Q_j = copy.deepcopy(Q_j_orig)
    if d_plus in Q_j:
        Q_j.remove(d_plus)
    _mw = {i: mw(i, d_plus, Q_j) for i in range(len(d_plus))}
    while Q_j and _mw:
        attr_selected = max(_mw, key=_mw.get)
        t.add(attr_selected)
        _mw.pop(attr_selected)
        for_deletion = []
        for d_minus in Q_j:
            if attr_selected in distinguishing(d_plus, d_minus):
                for_deletion.append(d_minus)
        for d in for_deletion:
            Q_j.remove(d)
    return t

In [170]:
def upravl(Q_i, Q_j):
    T = set()
    for d_plus in Q_i:
        mt = poisk(d_plus, Q_j)
        if mt not in T:
            T = T.union(mt)
    return T

In [171]:
upravl(train_dict['a'], train_dict['a'])

{0, 2, 3, 6}

In [204]:
QMT = defaultdict(dict)
for i in train_dict:
    for j in train_dict:
        QMT[i][j] = upravl(train_dict[i], train_dict[j])

In [205]:
QMT

defaultdict(dict,
            {'a': {'a': {0, 2, 3, 6},
              'b': {0, 1},
              'c': {0, 2},
              'd': {0},
              'e': {0},
              'x': {0}},
             'b': {'a': {0, 1},
              'b': {0, 2, 3},
              'c': {0, 1},
              'd': {0},
              'e': {0},
              'x': {0}},
             'c': {'a': {0, 1, 2},
              'b': {0, 1},
              'c': {1},
              'd': {0},
              'e': {0},
              'x': {0}},
             'd': {'a': {0},
              'b': {0},
              'c': {0},
              'd': {0, 1, 3},
              'e': {0, 1, 2},
              'x': {0, 1, 2}},
             'e': {'a': {0},
              'b': {0},
              'c': {0},
              'd': {0, 1, 2},
              'e': {0},
              'x': {0, 1, 4}},
             'x': {'a': {0},
              'b': {0},
              'c': {0},
              'd': {0, 1, 2},
              'e': {0, 4, 7},
              'x': {0}}})

In [199]:
X = [
    (3, 4, 3, 5, 3, 4, 2, 1, 1),
    (4, 4, 3, 3, 2, 4, 2, 3, 5)
]

In [201]:
train_dict['x'] = X

In [214]:
sim = {}

for x_c in QMT['x']:
    if x_c == 'x':
        continue

    sim[x_c] = 0
    print(f"Compare X with {x_c}")
    for c_c in QMT[x_c]:
        if c_c == 'x':
            continue
        print(f"{QMT['x'][c_c]} and {QMT[x_c][c_c]}")
        print(f"Intersection of {x_c} and {c_c} is {QMT[x_c][c_c].intersection(QMT['x'][c_c])}")
        sim[x_c] += len(QMT[x_c][c_c].intersection(QMT['x'][c_c]))


Compare X with a
{0} and {0, 2, 3, 6}
Intersection of a and a is {0}
{0} and {0, 1}
Intersection of a and b is {0}
{0} and {0, 2}
Intersection of a and c is {0}
{0, 1, 2} and {0}
Intersection of a and d is {0}
{0, 4, 7} and {0}
Intersection of a and e is {0}
Compare X with b
{0} and {0, 1}
Intersection of b and a is {0}
{0} and {0, 2, 3}
Intersection of b and b is {0}
{0} and {0, 1}
Intersection of b and c is {0}
{0, 1, 2} and {0}
Intersection of b and d is {0}
{0, 4, 7} and {0}
Intersection of b and e is {0}
Compare X with c
{0} and {0, 1, 2}
Intersection of c and a is {0}
{0} and {0, 1}
Intersection of c and b is {0}
{0} and {1}
Intersection of c and c is set()
{0, 1, 2} and {0}
Intersection of c and d is {0}
{0, 4, 7} and {0}
Intersection of c and e is {0}
Compare X with d
{0} and {0}
Intersection of d and a is {0}
{0} and {0}
Intersection of d and b is {0}
{0} and {0}
Intersection of d and c is {0}
{0, 1, 2} and {0, 1, 3}
Intersection of d and d is {0, 1}
{0, 4, 7} and {0, 1, 2}
In

In [215]:
sim

{'a': 5, 'b': 5, 'c': 4, 'd': 6, 'e': 7}